In [3]:
import numpy as np
import random
from copy import deepcopy
import logging

#random.seed(12)

class data_preprocessing:
    def __init__(self,instance_path):
        self.instance_path=instance_path
        
        self.info, self.flights = self.read_file(f_name=self.instance_path)
        self.number_of_areas,self.starting_airport=int(self.info[0][0]),self.info[0][1]
        
        
        self.flights_by_day_dict = self.flights_by_day(flight_list=self.flights)
        
        self.flights_by_day_dict=self.remove_duplicate(flights_by_day=self.flights_by_day_dict)
        
        self.list_days= [k for k in range(1,self.number_of_areas)]
        
        self.airports_by_area = self.get_airports_by_areas()
        self.area_to_explore=self.which_area_to_explore(airports_by_area=self.airports_by_area)
        self.area_by_airport=self.invert_dict(original_dict=self.airports_by_area)
        
        self.starting_area=self.associated_area_to_airport(airport=self.starting_airport)
        self.list_airports=self.get_list_of_airports()
        self.list_areas=list(self.airports_by_area.keys())
        self.areas_connections_by_day=self.possible_flights_from_zone_to_zone_specific_day()
        
    def read_file(self,f_name):
        dist = []
        line_nu = -1
        with open(f_name) as infile:
            for line in infile:
                line_nu += 1
                if line_nu == 0:
                    index = int(line.split()[0]) * 2 + 1
                if line_nu >= index:
                    temp = line.split()
                    temp[2] = int(temp[2])
                    temp[3] = int(temp[3])
                    dist.append(temp)
                else:
                    dist.append(line.split())
            info = dist[:int(dist[0][0])*2+1]
            flights = dist[int(dist[0][0])*2+1:]
        return info, flights
    
    def flights_by_day(self,flight_list):
        # Create an empty dictionary to hold flights organized by day
        flights_by_day = {}

        # Iterate over each flight in the input list
        for flight in flight_list:
            # Extract the day from the flight entry
            day = flight[2]

            # Create a flight entry without the day
            flight_without_day = flight[:2] + flight[3:]

            # Add the flight to the corresponding day in the dictionary
            if day not in flights_by_day:
                flights_by_day[day] = []
            flights_by_day[day].append(flight_without_day)
            
        return flights_by_day
    
    def flights_from_airport(self,flights_by_day, from_airport, considered_day):
        flights_from_airport = []
        for day, flights in flights_by_day.items():
            if day==considered_day:
                for flight in flights:
                    if flight[0] == from_airport:
                        flights_from_airport.append(flight)
                return flights_from_airport
            else:
                return None

    def invert_dict(self,original_dict):
        inverted_dict = {}
        for key, value_list in original_dict.items():
            for value in value_list:
                if value in inverted_dict:
                    inverted_dict[value].append(key)
                else:
                    inverted_dict[value] = key
        return inverted_dict

    def get_cost(self, day, from_airport, to_airport):
        # Retrieve flights for the specified day and day 0
        flights_day = self.flights_by_day_dict.get(day, [])
        flights_day_0 = self.flights_by_day_dict.get(0, [])
        
        # Find the cost for the specified day
        cost_day = next(
            (flight[2]
            for flight in flights_day
            if flight[0] == from_airport and flight[1] == to_airport),
            float('inf')
        )
        
        # Find the cost for day 0
        cost_day_0 = next(
            (flight[2]
            for flight in flights_day_0
            if flight[0] == from_airport and flight[1] == to_airport),
            float('inf')
        )
        
        # Return the minimum cost if either exists, otherwise inf
        if cost_day == float('inf') and cost_day_0 == float('inf'):
            return float('inf')
        
        return min(cost_day, cost_day_0)

    def possible_flights_from_zone_to_zone_specific_day(self):
        areas_connections_by_day = {}

        for day, flights in self.flights_by_day_dict.items():
            areas_connections_list = []

            for flight in flights:
                connection = f"{self.area_by_airport.get(flight[0])} to {self.area_by_airport.get(flight[1])}"
                if connection not in areas_connections_list:
                    areas_connections_list.append(connection)

            areas_connections_by_day[day] = areas_connections_list

        return areas_connections_by_day

    def get_airports_by_areas(self):
        area_num = int(self.info[0][0])
        return {f"{i}": self.info[2+i * 2] for i in range(0, area_num)}
    
    def get_list_of_airports(self):
        unique_airports = set()

        # Iterate through each sublist and add elements to the set
        for sublist in self.airports_by_area.values():
            for airport in sublist:
                unique_airports.add(airport)
        
        return list(unique_airports)
                    
    def associated_area_to_airport(self,airport):
        return next(
            (
                area
                for area, airports in self.airports_by_area.items()
                if airport in airports
            ),
            "Airport not found",
        ) 
    
    def remove_duplicate(self,flights_by_day):
        for day, flights in flights_by_day.items():
            unique_flights = {}
            for flight in flights:
                flight_key = (flight[0], flight[1])
                if flight_key not in unique_flights:
                    unique_flights[flight_key] = flight
                else:
                    if flight[2] < unique_flights[flight_key][2]:
                        #print(flight[2],unique_flights[flight_key][2])
                        unique_flights[flight_key] = flight
                flights_by_day[day] = list(unique_flights.values())
        return flights_by_day
    
    def possible_flights_from_an_airport_at_a_specific_day(self,day,from_airport):
        daily_flights = self.flights_by_day_dict.get(day, [])
        
        flights_from_airport = []
        for flight in daily_flights:
            if flight[0] == from_airport:
                
                flights_from_airport.append([flight[1], flight[2]])

        return flights_from_airport
    
    def possible_flights_from_an_airport_at_a_specific_day_with_previous_areas(self,day,from_airport, visited_areas):
        daily_flights = self.flights_by_day_dict.get(day, [])+ self.flights_by_day_dict.get(0, [])
        flights_from_airport = []
        for flight in daily_flights:
            #print(self.associated_area_to_airport(airport=flight[0]))
            if (flight[0] == from_airport) and (self.associated_area_to_airport(airport=flight[1]) not in visited_areas):
                
                flights_from_airport.append([flight[1], flight[2]])

        return flights_from_airport
    
    def which_area_to_explore(self,airports_by_area):
        return list({key: len(value) for key, value in airports_by_area.items() if len(value) > 1})
    
class Node:
    def __init__(self, state, parent=None):
        self.state = state  # State is a dictionary representing the current situation
        self.parent = parent  # Parent node
        self.children = []  # List of child nodes
        self.visit_count = 0  # Number of times this node has been visited
        self.total_cost = 0  # Total cost accumulated in simulations from this node

    def add_child(self, child_state):
        child_node = Node(child_state, self)
        self.children.append(child_node)
        return child_node

    def is_fully_expanded(self):
        if self.parent is None:
            return False
        return len(self.children) > 0 and all(child.visit_count > 0 for child in self.children)
    
    def best_child(self, c_param=1.41):
        epsilon = 0

        visited_children = [child for child in self.children if (child.visit_count > 0)]
        unvisited_children = [child for child in self.children if child.visit_count == 0]

        sorted_children = sorted(visited_children, key=lambda child: child.total_cost / (child.visit_count + epsilon))
        scores = {child: rank + 1 for rank, child in enumerate(sorted_children)}
        total_scores = sum(scores.values())

        def normalized_score(child):
            return scores[child] / total_scores

        choices_weights = [
            normalized_score(child) + c_param * ( np.log(self.visit_count + 1) / (child.visit_count + epsilon)) ** 0.5
            for child in visited_children
        ]

        print(f"UCT score {choices_weights}")
        best_child_node = self.children[np.argmin(choices_weights)]
        print(f"Selected Node: {best_child_node.state}, , Visit Count: {best_child_node.visit_count}, Total Cost: {best_child_node.total_cost}")  # Log the selected node
        print("Children picked on UCT critera")
        return best_child_node
    
    def update(self, result):
        self.visit_count += 1
        self.total_cost += result
        
class MCTS(data_preprocessing):
    def __init__(self, instance):
        super().__init__(instance_path=instance)
        self.logger = self.configure_logging()
        self.root=Node(self.initialise_root_node())
        self.best_leaf = None
        self.best_leaf_cost = float('inf')
        self.search()
        

    def configure_logging(self):
        log_file=f'{self.instance_path}.log'
        # Configure the logger
        logging.basicConfig(
            level=logging.DEBUG,  # Set the log level to DEBUG to capture all types of logs
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(log_file, mode='w'),  # 'w' to overwrite the log file each run, 'a' to append
                logging.StreamHandler()  # Optional: to also print logs to the console
            ]
        )
        logger = logging.getLogger(__name__)
        return logger

    def initialise_root_node(self):
        return {
        'current_day': 1,
        'current_airport': self.starting_airport,
        'remaining_zones': [x for x in self.list_areas if x != self.starting_area],  # Exclude the starting area
        'visited_zones': [self.starting_area],  # Exclude the starting area
        'total_cost': 0,
        'path': [self.starting_airport]
        }
                
    def expand_node(self, node):
        actions = self.possible_flights_from_an_airport_at_a_specific_day_with_previous_areas(
            node.state['current_day'], node.state['current_airport'], node.state['visited_zones'])

        self.logger.debug('\nExpansion')

        for action in actions:
            new_state = self.transition_function(node.state, action)
            child_node = node.add_child(new_state)
            self.logger.info(f"Children: {child_node.state}, Visit count: {child_node.visit_count}, Total cost: {child_node.total_cost}")
        self.logger.debug('\nEnd expansion')
        
    def transition_function(self, state, action):
        new_state = deepcopy(state)
        new_state['current_day'] += 1
        new_state['current_airport'] = action[0]
        new_state['total_cost'] += action[1]
        new_state['path'].append(action[0])
        new_state['remaining_zones'].remove(self.associated_area_to_airport(airport=new_state['current_airport']))
        new_state['visited_zones'].append(self.associated_area_to_airport(airport=action[0]))
        return new_state

    def random_policy(self, state):
        actions = self.possible_flights_from_an_airport_at_a_specific_day_with_previous_areas(
            day=state['current_day'], from_airport=state['current_airport'], visited_areas=state['visited_zones']
        )
        if not actions:
            return None
        return random.choice(actions)
    
    def heuristic_policy(self, state):
        actions = self.possible_flights_from_an_airport_at_a_specific_day_with_previous_areas(
            day=state['current_day'], from_airport=state['current_airport'], visited_areas=state['visited_zones']
        )
        self.logger.info(f"Actions: {actions}")
        if not actions:
            return None
        # Select the action with the lowest cost
        best_action = min(actions, key=lambda x: x[1])
        self.logger.info(f"Chosen action based on heuristic policy: {best_action}")
        return best_action
    
    def select(self, node):
        while True:
            if not node.children:
                self.expand_node(node)
                
                if not node.children:  # No more children can be expanded
                        return node
                return random.choice(node.children)
            
            unvisited_children = self.get_unvisited_children(node)
            if unvisited_children:
                self.logger.info(f"Unvisited children picked randomly")
                return random.choice(unvisited_children)
            
            node = node.best_child()
            self.logger.info(f"The best node has been chosen on UCT {node.state}")


    def get_unvisited_children(self, node):
        queue = [node]
        unvisited_children = []
        while queue:
            current_node = queue.pop(0)
            for child in current_node.children:
                if child.visit_count == 0:
                    unvisited_children.append(child)
                else:
                    queue.append(child)

        return unvisited_children

    def simulate(self, node):
        current_simulation_state = deepcopy(node.state)
        self.logger.info(f"Selected node for simulation {current_simulation_state}")
        while current_simulation_state['current_day'] != self.number_of_areas:
            action = self.heuristic_policy(state=current_simulation_state)
            if action is None:  # No valid actions available
                break

            current_simulation_state = self.transition_function(current_simulation_state, action)
            self.logger.info(f'Current simulation state {current_simulation_state}')
        
        flights_to_go_back_initial_zone = self.possible_flights_from_an_airport_at_a_specific_day_with_previous_areas(
            day=current_simulation_state['current_day'], from_airport=current_simulation_state['current_airport'], 
            visited_areas=current_simulation_state['visited_zones'][1:])
        
        min_price = 100000
        final_airport = None
        
        for flight in flights_to_go_back_initial_zone:
            if flight[1] < min_price:
                min_price = flight[1]
                final_airport = flight[0]
        self.logger.info(f"Last flight to {final_airport} at the cost: {min_price}")
        print(f"{min_price}, {final_airport}")
        current_simulation_state['total_cost'] += min_price

        if current_simulation_state['total_cost'] < self.best_leaf_cost:
            self.best_leaf = current_simulation_state
            self.best_leaf_cost = current_simulation_state['total_cost']
        
        #print(f"Simulation result: {current_simulation_state}")
        return current_simulation_state['total_cost']

    def backpropagate(self, node, cost):
        while node is not None:
 
            node.update(cost)

            self.logger.info(f"Backpropagating Node: {node.state}, Visit Count: {node.visit_count}, Total Cost: {node.total_cost}")

            node = node.parent

    def expand_and_select(self, node):
        if not node.is_fully_expanded():
            self.expand_node(node)
        unvisited_children = self.get_unvisited_children(node)
        if unvisited_children:
            return random.choice(unvisited_children)
        return node.best_child()
        
    def search(self):
        #for _ in range(self.iterations):
        while True:
            node_to_explore = self.select(self.root)
            if len(node_to_explore.state['remaining_zones'])==0:
                node_to_explore = self.select(self.root)
                cost = self.simulate(node_to_explore)
                self.backpropagate(node_to_explore, cost)
                return
                
            if node_to_explore.is_fully_expanded():
                node_to_explore = self.expand_and_select(node_to_explore)
            cost = self.simulate(node_to_explore)
            self.backpropagate(node_to_explore, cost)
        
    def collect_all_nodes(self):
        nodes = []
        queue = [self.root]
        while queue:
            node = queue.pop(0)
            nodes.append(node)
            queue.extend(node.children)
        return nodes

    def display_all_nodes(self, nodes):
        for node in nodes:
            print(f"State: {node.state}, Visit Count: {node.visit_count}, Total Cost: {node.total_cost}")
        
            
            self.logger.info(f"State: {node.state}, Visit Count: {node.visit_count}, Total Cost: {node.total_cost}")

instance_path = '/Users/adslv/Documents/LU/Term 3/Dissertation/Code/Flight connections dataset/3.in'
#data_processor = data_preprocessing(instance_path=instance_path)

mcts = MCTS(instance=instance_path)
print(mcts.best_leaf)
print('\n')
#mcts.display_all_nodes(nodes=mcts.collect_all_nodes())



2024-07-19 14:35:46,575 - __main__ - DEBUG - 
Expansion
2024-07-19 14:35:46,575 - __main__ - INFO - Children: {'current_day': 2, 'current_airport': 'MZ1', 'remaining_zones': ['0', '1', '2', '3', '4', '5', '7', '9', '10', '11', '12'], 'visited_zones': ['8', '6'], 'total_cost': 1390, 'path': ['GDN', 'MZ1']}, Visit count: 0, Total cost: 0
2024-07-19 14:35:46,576 - __main__ - INFO - Children: {'current_day': 2, 'current_airport': 'KRK', 'remaining_zones': ['0', '1', '2', '3', '4', '6', '7', '9', '10', '11', '12'], 'visited_zones': ['8', '5'], 'total_cost': 2361, 'path': ['GDN', 'KRK']}, Visit count: 0, Total cost: 0
2024-07-19 14:35:46,576 - __main__ - INFO - Children: {'current_day': 2, 'current_airport': 'SA1', 'remaining_zones': ['0', '1', '2', '3', '4', '5', '6', '7', '10', '11', '12'], 'visited_zones': ['8', '9'], 'total_cost': 2242, 'path': ['GDN', 'SA1']}, Visit count: 0, Total cost: 0
2024-07-19 14:35:46,577 - __main__ - INFO - Children: {'current_day': 2, 'current_airport': 'KTW',

673, OSP
1740, PM1
2046, PM1
621, PM1
1740, PM1
621, PM1
621, PM1
673, OSP
621, PM1
1740, PM1
1356, OSP
673, OSP
621, PM1
621, PM1
673, OSP


2024-07-19 14:35:46,801 - __main__ - INFO - Actions: [['SZZ', 1935], ['ZC1', 1640], ['WRO', 1830], ['IEG', 1862], ['ZC2', 2023], ['OSZ', 1517], ['ZC3', 1551], ['DL1', 2168], ['BXP', 1107], ['LB1', 1113]]
2024-07-19 14:35:46,801 - __main__ - INFO - Chosen action based on heuristic policy: ['BXP', 1107]
2024-07-19 14:35:46,802 - __main__ - INFO - Current simulation state {'current_day': 10, 'current_airport': 'BXP', 'remaining_zones': ['0', '3', '12'], 'visited_zones': ['8', '7', '5', '9', '4', '11', '1', '6', '10', '2'], 'total_cost': 7303, 'path': ['GDN', 'PD1', 'KRK', 'SA1', 'LD1', 'WE2', 'KJ1', 'WMI', 'SZY', 'BXP']}
2024-07-19 14:35:46,802 - __main__ - INFO - Actions: [['SZZ', 2816], ['ZC1', 2570], ['WRO', 2093], ['IEG', 2457], ['ZC2', 2870], ['OSZ', 2495], ['ZC3', 2418], ['DL1', 2467]]
2024-07-19 14:35:46,802 - __main__ - INFO - Chosen action based on heuristic policy: ['WRO', 2093]
2024-07-19 14:35:46,802 - __main__ - INFO - Current simulation state {'current_day': 11, 'current_air

673, OSP
621, PM1
621, PM1
621, PM1
621, PM1
621, PM1
1356, OSP
621, PM1
673, OSP
2046, PM1
621, PM1
621, PM1
621, PM1
628, PM1


2024-07-19 14:35:47,005 - __main__ - INFO - Chosen action based on heuristic policy: ['BXP', 1107]
2024-07-19 14:35:47,006 - __main__ - INFO - Current simulation state {'current_day': 7, 'current_airport': 'BXP', 'remaining_zones': ['0', '3', '5', '7', '9', '12'], 'visited_zones': ['8', '1', '11', '4', '6', '10', '2'], 'total_cost': 3795, 'path': ['GDN', 'BZG', 'WE2', 'LD2', 'WMI', 'SZY', 'BXP']}
2024-07-19 14:35:47,006 - __main__ - INFO - Actions: [['KRK', 1506], ['SA1', 1640], ['KTW', 1557], ['PD1', 1057], ['RZE', 1083], ['SZZ', 2816], ['ZC1', 2570], ['WRO', 2093], ['IEG', 2457], ['ZC2', 2870], ['SA2', 1794], ['OSZ', 2495], ['ZC3', 2418], ['DL1', 2467]]
2024-07-19 14:35:47,006 - __main__ - INFO - Chosen action based on heuristic policy: ['PD1', 1057]
2024-07-19 14:35:47,007 - __main__ - INFO - Current simulation state {'current_day': 8, 'current_airport': 'PD1', 'remaining_zones': ['0', '3', '5', '9', '12'], 'visited_zones': ['8', '1', '11', '4', '6', '10', '2', '7'], 'total_cost': 4

...

1918, PM1
628, PM1
697, PM1
673, OSP
673, OSP
628, PM1
1918, PM1
1918, PM1
673, OSP
1918, PM1
628, PM1
1918, PM1
1918, PM1
673, OSP
1918, PM1
1918, PM1
UCT score [3.1917174333609926, 3.2352780394215985, 3.227702281845841, 3.2258083424519017, 3.237171978815538, 3.2030810697246292, 3.240959857603417, 3.222020463664023, 3.208762887906447, 3.193611372754932, 3.2333841000276595, 3.23149016063372, 3.184141675785235, 3.1955053121488715, 3.2239144030579623, 3.2295962212397806, 3.20118713033069, 3.2049750091185683, 3.1822477363912958, 3.2068689485125077, 3.216338645482205, 3.189823493967053, 3.187929554573114, 3.2106568273003866, 0.316814268067797, 3.2390659182094774, 3.2201265242700834, 3.212550766694326, 3.1992931909367504, 3.2144447060882655, 3.197399251542811, 3.1860356151791747]
Selected Node: {'current_day': 2, 'current_airport': 'SZY', 'remaining_zones': ['0', '1', '2', '3', '4', '5', '6', '7', '9', '11', '12'], 'visited_zones': ['8', '10'], 'total_cost': 903, 'path': ['GDN', 'SZY']}, , 

2024-07-19 14:35:48,864 - __main__ - INFO - Chosen action based on heuristic policy: ['KJ1', 1332]
2024-07-19 14:35:48,864 - __main__ - INFO - Current simulation state {'current_day': 12, 'current_airport': 'KJ1', 'remaining_zones': ['12'], 'visited_zones': ['8', '10', '6', '4', '2', '7', '5', '9', '11', '3', '0', '1'], 'total_cost': 7877, 'path': ['GDN', 'SZY', 'WMI', 'LD3', 'LB1', 'PD1', 'KRK', 'SA1', 'POZ', 'IEG', 'DL1', 'KJ1']}
2024-07-19 14:35:48,864 - __main__ - INFO - Actions: [['SZZ', 1189], ['ZC1', 967], ['ZC2', 1242], ['OSZ', 939], ['ZC3', 793]]
2024-07-19 14:35:48,864 - __main__ - INFO - Chosen action based on heuristic policy: ['ZC3', 793]
2024-07-19 14:35:48,865 - __main__ - INFO - Current simulation state {'current_day': 13, 'current_airport': 'ZC3', 'remaining_zones': [], 'visited_zones': ['8', '10', '6', '4', '2', '7', '5', '9', '11', '3', '0', '1', '12'], 'total_cost': 8670, 'path': ['GDN', 'SZY', 'WMI', 'LD3', 'LB1', 'PD1', 'KRK', 'SA1', 'POZ', 'IEG', 'DL1', 'KJ1', 'Z

673, OSP
673, OSP
673, OSP
1740, PM1
UCT score [3.2157529689443916, 3.2593135750049975, 3.25173781742924, 3.2498438780353007, 3.261207514398937, 3.2271166053080282, 3.264995393186816, 3.246055999247422, 3.232798423489846, 3.217646908338331, 3.2574196356110585, 3.255525696217119, 3.208177211368634, 3.2195408477322704, 3.2479499386413613, 3.2536317568231796, 3.225222665914089, 3.2290105447019672, 3.2062832719746948, 3.2309044840959067, 3.240374181065604, 3.213859029550452, 3.211965090156513, 3.2346923628837856, 0.305843285511049, 3.2631014537928764, 3.2441620598534824, 3.236586302277725, 3.2233287265201493, 3.2384802416716645, 3.22143478712621, 3.2100711507625737]
Selected Node: {'current_day': 2, 'current_airport': 'SZY', 'remaining_zones': ['0', '1', '2', '3', '4', '5', '6', '7', '9', '11', '12'], 'visited_zones': ['8', '10'], 'total_cost': 903, 'path': ['GDN', 'SZY']}, , Visit Count: 143, Total Cost: 1433625
Children picked on UCT critera
UCT score [3.147356266321865, 3.19171110503154

2024-07-19 14:35:49,093 - __main__ - INFO - Backpropagating Node: {'current_day': 1, 'current_airport': 'GDN', 'remaining_zones': ['0', '1', '2', '3', '4', '5', '6', '7', '9', '10', '11', '12'], 'visited_zones': ['8'], 'total_cost': 0, 'path': ['GDN']}, Visit Count: 193, Total Cost: 1900407
2024-07-19 14:35:49,093 - __main__ - INFO - Unvisited children picked randomly
2024-07-19 14:35:49,094 - __main__ - INFO - Selected node for simulation {'current_day': 11, 'current_airport': 'OSZ', 'remaining_zones': ['1', '11'], 'visited_zones': ['8', '10', '6', '4', '2', '7', '5', '9', '0', '3', '12'], 'total_cost': 6654, 'path': ['GDN', 'SZY', 'WMI', 'LD3', 'LB1', 'PD1', 'KRK', 'SA1', 'WRO', 'IEG', 'OSZ']}
2024-07-19 14:35:49,094 - __main__ - INFO - Actions: [['POZ', 894], ['BZG', 756], ['WE1', 951], ['KJ1', 939], ['WE2', 1036]]
2024-07-19 14:35:49,095 - __main__ - INFO - Chosen action based on heuristic policy: ['BZG', 756]
2024-07-19 14:35:49,095 - __main__ - INFO - Current simulation state {'c

982, PM1
673, OSP
UCT score [3.2507184724407505, 3.2942790785013565, 3.286703320925599, 3.2848093815316597, 3.296173017895296, 3.262082108804387, 3.299960896683175, 3.281021502743781, 3.267763926986205, 3.25261241183469, 3.2923851391074175, 3.290491199713478, 3.243142714864993, 3.2545063512286294, 3.2829154421377202, 3.2885972603195386, 3.2601881694104478, 3.263976048198326, 3.2412487754710537, 3.2658699875922657, 3.275339684561963, 3.248824533046811, 3.246930593652872, 3.2696578663801446, 0.2908299032998938, 3.2980669572892354, 3.2791275633498413, 3.271551805774084, 3.2582942300165083, 3.2734457451680234, 3.256400290622569, 3.2450366542589326]
Selected Node: {'current_day': 2, 'current_airport': 'SZY', 'remaining_zones': ['0', '1', '2', '3', '4', '5', '6', '7', '9', '11', '12'], 'visited_zones': ['8', '10'], 'total_cost': 903, 'path': ['GDN', 'SZY']}, , Visit Count: 164, Total Cost: 1611854
Children picked on UCT critera
UCT score [3.1901160919680973, 3.234470930677775, 3.232454801645